In [1]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import plotly.graph_objects as go

In [2]:
#Loops through USA States on Google open Data by location key and downloads data concurrently 
#Prints error and error code if iteration is failed
def googleDataCall(state):
    try:
        url = f"https://storage.googleapis.com/covid19-open-data/v3/location/US_{state}.csv"
        state_data = pd.read_csv(url)
        return state_data
    except Exception as e:
        print(f"Failed to download data for state: {state}. Error: {e}")


states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
#Makes data extraction concurrent instead of simultaneous
with ThreadPoolExecutor() as executor:
    googleDl = list(executor.map(googleDataCall, states))
    googleData = pd.concat(googleDl, ignore_index=True)
#This is the data filter, removing columns deemed unnecessary 
unusedColumns = list(googleData.columns[2:7]) + list(googleData.columns[47:65]) + list(googleData.columns[67:75]) + list(googleData.columns[108:614])
usaData = googleData.drop(unusedColumns, axis=1)

  

In [12]:
#usaData

In [4]:
DeadDf = usaData[['date', 'cumulative_confirmed', 'cumulative_persons_vaccinated', 'cumulative_deceased', 'subregion1_code','stringency_index']]
cleanDeadDf = DeadDf.copy() 
cleanDeadDf = cleanDeadDf.interpolate()
cleanDeadDf.fillna(0 , inplace=True)


In [15]:
fig = go.Figure(data=go.Choropleth(
    locations=cleanDeadDf['subregion1_code'], 
    z = cleanDeadDf['cumulative_deceased'].astype(float),
    locationmode = 'USA-states', 
    colorscale = 'Reds',
    colorbar_title = "COVID Death",
))

fig.update_layout(
    title_text = 'COVID Death Map JAN2020-MAR2022',
    geo_scope='usa',
)
fig.write_image("images/USAcovidDeathMap.png")

In [16]:
fig = go.Figure(data=go.Choropleth(
    locations=cleanDeadDf['subregion1_code'], 
    z = cleanDeadDf['cumulative_persons_vaccinated'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'Reds',
    colorbar_title = "COVID Vaccinations",
))

fig.update_layout(
    title_text = 'COVID Vaccination Map JAN2020-MAR2022',
    geo_scope='usa',
)
fig.write_image("images/USAcovidVaccMap.png")

Birth State Map Creation

In [17]:
stateFert = pd.read_csv("data/data-table.csv")

In [18]:
stateFert2021 = stateFert[0:50]
stateFert2020 = stateFert[50:100]
stateFert2019 = stateFert[100:151]

In [19]:
fig = go.Figure(data=go.Choropleth(
    locations=stateFert2019['STATE'], 
    z = stateFert2019['BIRTHS'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'algae',
    colorbar_title = "2019 Births",
))

fig.update_layout(
    title_text = '2019 Births by State',
    geo_scope='usa',
)
fig.write_image("images/USA2019BirthMap.png")

In [20]:
fig = go.Figure(data=go.Choropleth(
    locations=stateFert2020['STATE'], 
    z = stateFert2020['BIRTHS'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'algae',
    colorbar_title = "2020 Births",
))

fig.update_layout(
    title_text = '2020 Births by State',
    geo_scope='usa',
)
fig.write_image("images/USA2020BirthMap.png")

In [21]:
fig = go.Figure(data=go.Choropleth(
    locations=stateFert2021['STATE'], 
    z = stateFert2021['BIRTHS'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'algae',
    colorbar_title = "2021 Births",
))

fig.update_layout(
    title_text = '2021 Births by State',
    geo_scope='usa',
)
fig.write_image("images/USA2021BirthMap.png")